## Analyzing Covid 19 Data, Mortality, Food security

This notebook analyzes Covid 19 data collected by the New York Times.

https://github.com/nytimes/covid-19-data

Put this notebook in the root of the repository before running the code.

Be sure to run "git pull origin master" before analyzing the data to pull the most recent data.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import os
path = os.getcwd()
path

'C:\\Users\\ck24\\ACE_592\\covid-19-data'

In [7]:
path[:-13]+r'foodsecurity_census\dec19pub.csv'covid-19-datafoosecurity_census

'C:\\Users\\ck24\\ACE_592\\foodsecurity_census\\dec19pub.csv'

In [12]:
# Read data
df_fs19 = pd.read_csv(path+r'\foosecurity_census\dec19pub.csv')#.sort_values(['state','date'],ascending=[True, True])

In [14]:
len(df_fs19.columns)

510

In [66]:
df_fs19.columns[90:100]

Index(['PEHRUSLT', 'PEHRWANT', 'PEHRRSN1', 'PEHRRSN2', 'PEHRRSN3', 'PUHROFF1',
       'PUHROFF2', 'PUHROT1', 'PUHROT2', 'PEHRACT1'],
      dtype='object')

In [16]:
len(df_fs19)

138964

In [189]:
subset=['HRHHID',#Houshold ID
        'GCFIP', #state ecode
        'GCTCO', #county code
        'PRPERTYP', #Type (1:child, 2:adult, 3: adulte armed force,
        'HRINTSTA', ##HH interview status
        'HWHHWGT','HHSUPWGT', #HH weight
        'HRNUMHOU', #HH number
        'HRHTYPE', #0:noninterview,
        'PRTAGE', #Persons Age
        'PEMARITL',#marital status
        'PESEX',#Sex
        'PEEDUCA',#education
        'PTDTRACE',#race,
        'PEHSPNON',#hispanic
        'HRPOOR',#poor
        'HEFAMINC',#family incom
        'HRFS12M1',#summary of fs
        'HRFS12MD',#summary detail of FS
        'HRFS12MC',#summary 
        'HRFS30D8',#30Day recall adult detail
        'HRFS30DE',#adult FS Rasch Scale Score 30Day recall]
        ]
         #+[x for x in df_fs19 if x.startswith('HRFS')]
#fips=[x for x in df_fs19.columns if 'WGT' in x]
#subset2 =[x for x in df_fs19 if x.startswith('G')]

In [190]:
df=df_fs19[subset].replace([-1,-9],np.nan)

#df['HRFS12MD']=df['HRFS12MD'].replace([-1,-9],np.nan)
df['HRFS12MD']=df['HRFS12MD'].replace([1,2],0)
df['HRFS12MD']=df['HRFS12MD'].replace([3,4],1)
df['HRFS12MD'].value_counts()

df['HRPOOR']=df['HRPOOR'].replace(2,0)
df['HRPOOR'].value_counts()

df['PEHSPNON']=df['PEHSPNON'].replace(2,0)
df['PEHSPNON'].value_counts()

df['PESEX']=df['PESEX'].replace(2,0)
df['PESEX'].value_counts()

#df['PESEX']=df['PESEX'].replace(2,0)
df['PEEDUCA'].value_counts()
df['PRTAGE'].sort_values(ascending=True)#.value_counts()
df['HEFAMINC'].value_counts()

16.0    17998
15.0    17927
14.0    15779
13.0    12065
12.0     9552
11.0     8970
9.0      6267
10.0     6041
7.0      5071
8.0      4936
6.0      3574
4.0      2441
1.0      2315
5.0      2237
3.0      1647
2.0      1112
Name: HEFAMINC, dtype: int64

In [195]:
import statsmodels.api as sm
import numpy as np

<module 'statsmodels.api' from 'C:\\Users\\ck24\\.conda\\envs\\ace592\\lib\\site-packages\\statsmodels\\api.py'>

In [216]:
#deselect noninterview
intvw=df[df['HRHTYPE']!=0]
#adult_intvw=intvw[intvw['PRPERTYP']!=1]

HHID_group = intvw.groupby("HRHHID").agg('mean')
state_group = intvw.groupby("GCFIP").agg('mean')

subset_reg=['HRFS12MD','HRPOOR','PEHSPNON','PEEDUCA','PRTAGE','HEFAMINC','HRNUMHOU']
HHID_reg=HHID_group[subset_reg]
HHID_reg_dropna=HHID_reg.dropna()

state_reg=state_group[subset_reg]

In [217]:
state_reg

,HRFS12MD,HRPOOR,PEHSPNON,PEEDUCA,PRTAGE,HEFAMINC,HRNUMHOU
GCFIP,,,,,,,
1,0.144509,0.324928,0.039183,40.214901,41.546895,11.315131,2.994998
2,0.096136,0.233742,0.053363,40.421292,38.247076,12.339912,3.372807
4,0.129032,0.314516,0.310723,40.269465,40.192519,11.810973,3.273317
5,0.109318,0.294204,0.092908,40.145445,40.715889,11.206463,3.069120
6,0.099305,0.289855,0.377032,40.232520,40.027022,12.229783,3.484237
8,0.113208,0.233962,0.189459,40.892400,39.870370,12.352564,3.141026
9,0.105563,0.202550,0.146134,40.595787,43.183517,12.627867,3.105353
10,0.085526,0.254260,0.090909,40.622841,42.815789,11.859649,3.127592
11,0.053948,0.178599,0.083840,42.146376,39.404010,13.131227,2.688943


In [222]:
Y = HHID_reg_dropna['HRFS12MD']
X = HHID_reg_dropna[['HRPOOR','PEHSPNON','PRTAGE','HEFAMINC','HRNUMHOU']]
X=sm.add_constant(X)
model=sm.OLS(Y,X)
results=model.fit()
results.summary()


log_reg = sm.Logit(Y, X).fit()
log_reg.summary()


Optimization terminated successfully.
         Current function value: 0.273914
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               HRFS12MD   No. Observations:                31218
Model:                          Logit   Df Residuals:                    31212
Method:                           MLE   Df Model:                            5
Date:                Fri, 30 Apr 2021   Pseudo R-squ.:                  0.1519
Time:                        08:57:23   Log-Likelihood:                -8551.0
converged:                       True   LL-Null:                       -10082.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0890      0.109      0.818      0.413      -0.124       0.302
HRPOOR         0.8033      0.054     14.916      0.000       0.698       0.909
PEHSPNON       0.1564      0.059      2.647      0.008       0.041       0.272
PRTAGE        -0.0192      0.001    -15.799      0.000      -0.022      -0.017
HEFAMINC      -0.1741      0.007    -26.418      0.000      -0.187      -0.161
HRNUMHOU      -0.0064      0.017     -0.366      0.714      -0.041       0.028
==============================================================================
"""

In [218]:
#state
Y = state_reg['HRFS12MD']
X = state_reg[['HRPOOR','PEHSPNON','PEEDUCA','PRTAGE','HEFAMINC','HRNUMHOU']]
X=sm.add_constant(X)
model=sm.OLS(Y,X)
results=model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               HRFS12MD   R-squared:                       0.659
Model:                            OLS   Adj. R-squared:                  0.613
Method:                 Least Squares   F-statistic:                     14.18
Date:                Fri, 30 Apr 2021   Prob (F-statistic):           6.53e-09
Time:                        08:43:32   Log-Likelihood:                 144.56
No. Observations:                  51   AIC:                            -275.1
Df Residuals:                      44   BIC:                            -261.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2187      0.512      0.427      0.671      -0.813       1.250
HRPOOR         0.1859      0.099      1.881      0.067      -0.013       0.385
PEHSPNON      -0.0037      0.024     -0.152      0.880      -0.053       0.046
PEEDUCA       -0.0031      0.012     -0.261      0.795      -0.027       0.021
PRTAGE         0.0011      0.002      0.693      0.492      -0.002       0.004
HEFAMINC      -0.0127      0.011     -1.208      0.233      -0.034       0.009
HRNUMHOU       0.0207      0.019      1.085      0.284      -0.018       0.059
==============================================================================
Omnibus:                        0.517   Durbin-Watson:                   1.808
Prob(Omnibus):                  0.772   Jarque-Bera (JB):                0.630
Skew:                          -0.040   Prob(JB):                        0.730
Kurtosis:                       2.461   Cond. No.                     1.41e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.41e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

,HRPOOR,PEHSPNON,PEEDUCA,PRTAGE,HEFAMINC,HRNUMHOU
HRHHID,,,,,,
4792610319,0.000000,0.0,40.500000,56.000000,13.0,2.000000
4795110719,NaN,0.0,40.000000,51.400000,7.6,2.600000
8177510065,0.166667,0.0,39.600000,39.666667,14.5,4.333333
8178510165,0.000000,0.0,42.500000,24.000000,15.0,4.000000
10415001537,0.000000,0.0,38.000000,63.000000,10.0,2.000000
...,...,...,...,...,...,...
999006781019701,1.000000,1.0,36.000000,71.000000,5.0,2.000000
999220358571002,NaN,0.0,39.500000,63.500000,11.0,2.000000
999373301460408,0.400000,0.0,40.500000,29.800000,12.8,2.600000


In [121]:
#groupby operation
aggregations = {
    'PEEDUCA':'mean',
#    'date': lambda x: max(x) - 1
}
adult_intvw.groupby('HRHHID').agg(aggregations)

,PEEDUCA
HRHHID,
4792610319,40.500000
4795110719,40.000000
8177510065,39.600000
8178510165,42.500000
10415001537,38.000000
...,...
999006781019701,36.000000
999220358571002,39.500000
999373301460408,40.500000


In [118]:
wm = lambda x: np.average(x, weights=adult_intvw.loc[x.index, "HWHHWGT"])


In [117]:
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=df.loc[x.index, "adjusted_lots"])

# Define a dictionary with the functions to apply for a given column:
# the following is deprecated since pandas 0.20:
# f = {'adjusted_lots': ['sum'], 'price': {'weighted_mean' : wm} }
# df.groupby(["contract", "month", "year", "buys"]).agg(f)

# Groupby and aggregate with namedAgg [1]:
df.groupby(["contract", "month", "year", "buys"]).agg(adjusted_lots=("adjusted_lots", "sum"),  
                                                      price_weighted_mean=("price", wm))

NameError: name 'df' is not defined

In [70]:
df_fs19[subset2].GCFIP.sort_values(ascending=False) ## GCFIP = statecode
df_fs19[subset2].GCFIP.sort_values(ascending=False) ## GCFIP = statecode

138963    56
112380    56
112410    56
112409    56
112408    56
          ..
73114      1
73115      1
73116      1
74417      1
46082      1
Name: GCFIP, Length: 138964, dtype: int64

In [92]:
a=df_fs19[subset2].GCTCO.unique()
a[::1].sort()
a
#df_fs19[subset2].GCTCO.unique()

array([  0,   1,   3,   5,   7,   9,  11,  13,  15,  17,  19,  21,  23,
        25,  27,  29,  31,  33,  35,  37,  39,  41,  43,  45,  47,  49,
        51,  53,  55,  57,  59,  61,  63,  67,  69,  71,  73,  75,  77,
        79,  81,  83,  85,  86,  87,  89,  91,  93,  95,  97,  99, 101,
       103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 129,
       133, 135, 139, 141, 145, 147, 151, 153, 155, 159, 161, 163, 165,
       171, 173, 177, 179, 181, 189, 191, 215, 223, 251, 309, 423, 439,
       441, 479, 485, 510, 550, 700, 710, 760, 810], dtype=int64)

In [64]:
df_fs19[fips+['HRINTSTA','HHSUPWGT']]

,HWHHWGT,PWFMWGT,PWLGWGT,PWORWGT,PWSSWGT,PWVETWGT,PWCMPWGT,HHSUPWGT,PWSUPWGT,HRINTSTA,HHSUPWGT
0,4092969,4092969,6378844,0,4469015,4440048,4522419,3948531,4529284,1,3948531
1,4092969,4092969,5857526,0,4092969,4251416,4254478,3948531,3948531,1,3948531
2,4092969,5086587,7260335,0,5086587,5070891,5070891,3948531,5436219,1,3948531
3,4092969,5010140,0,0,5010140,0,0,3948531,5452299,1,3948531
4,17573768,17573768,24464728,65148420,17139976,16396811,16547278,20302067,22455089,1,20302067
...,...,...,...,...,...,...,...,...,...,...,...
138959,0,0,0,0,0,0,0,0,0,2,0
138960,0,0,0,0,0,0,0,0,0,2,0
138961,0,0,0,0,0,0,0,0,0,2,0
138962,0,0,0,0,0,0,0,0,0,2,0


In [33]:
df_fs19.HRINTSTA.value_counts()

1    117932
2     10579
3      9911
4       542
Name: HRINTSTA, dtype: int64

In [37]:
df_fs19.columns

Index(['HRHHID', 'HRMONTH', 'HRYEAR4', 'HURESPLI', 'HUFINAL', 'HULANGCODE',
       'HETENURE', 'HEHOUSUT', 'HETELHHD', 'HETELAVL',
       ...
       'HRFS30D1', 'HRFS30D2', 'HRFS30D3', 'HRFS30D4', 'HRFS30D5', 'HRFS30D6',
       'HRFS30D7', 'HRFS30D8', 'HRFS30D9', 'HRFS30DE'],
      dtype='object', length=510)

In [34]:
df_fs19[subset+['GTCO']]

KeyError: "['GTCO'] not in index"